<a href="https://colab.research.google.com/github/SanePai/Neural_Networks-Assignment-2/blob/main/Question_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import warnings
warnings.filterwarnings("ignore")

In [9]:
%cd "/content/drive/MyDrive/NNFL/Assignment 2"
!ls

/content/drive/.shortcut-targets-by-id/104/Assignment 2
assignment2.pdf  class2_images	class_label.mat  data5.mat     input.mat
class1_images	 class3_images	data55.xlsx	 input_a2.mat  label.mat


In [10]:
import numpy as np
from scipy.io import loadmat
from sklearn.preprocessing import normalize
import random
from sklearn.metrics import confusion_matrix as cm

In [11]:
f = loadmat('data5.mat')
D = f['x']
np.random.shuffle(D)


def init_data(data):
    X = np.array(data[ : , :-1], dtype = float)
    y = np.array(data[ : , -1], dtype = int)
    X = (X - X.mean(axis = 0))/X.std(axis = 0)
    return X, y

X, y = init_data(D)
X_train, y_train = X[ :int(0.7 * len(X))], y[ :int(0.7 * len(X))]
X_val, y_val = X[ int(0.7 * len(X)): ], y[ int(0.7 * len(X)): ]

alpha = 0.5

In [12]:
def sigmoid(x, derivative=False):
        if (derivative == True):
            return x * (1 - x)
        return 1 / (1 + np.exp(-x))

In [13]:
class NeuralNetwork(object):
    def __init__(self, sizes):
        
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.W = {}
        self.a = {}
        self.b = {}
        
        #Initialize Weights
        for i in range(1, self.num_layers):
            self.W[i] = np.random.randn(self.sizes[i-1], self.sizes[i])
            
        #Initialize biases
        for i in range(1, self.num_layers):
            self.b[i] = np.random.randn(self.sizes[i], 1)
        
        #Initialize activations
        for i in range(1, self.num_layers):
            self.a[i] = np.zeros([self.sizes[i], 1])
        
    #Forward pass
    def forward_pass(self, X):
        
        self.a[0] = X
        
        for i in range(1, self.num_layers):
            self.a[i] = sigmoid(np.dot(self.W[i].T, self.a[i-1]) + self.b[i])

        return self.a[self.num_layers-1] 
    
    #Backward pass/ Weight Update
    def backward_pass(self, X, Y, output):
        
        self.d = {}
        self.d_output = (Y - output) * sigmoid(output, derivative=True)
        self.d[self.num_layers-1] = self.d_output
        
        #Derivatives of the layers
        for i in range(self.num_layers-1, 1, -1):
            self.d[i-1] = np.dot(self.W[i], self.d[i]) * sigmoid(self.a[i-1], derivative=True)
        
        #Updating weights
        for i in range(1, self.num_layers-1):
            self.W[i] += alpha * np.dot(self.a[i-1], self.d[i].T)
            
        #Updating biases
        for i in range(1, self.num_layers-1):
            self.b[i] += alpha * self.d[i]

    def train(self, X, Y):
        X = np.reshape(X, (len(X), 1))
        output = self.forward_pass(X)
        self.backward_pass(X, Y, output)
    
    def get_W(self):
        return self.W
    
    def load_W(self, W):
        self.W = W
    
    def get_a(self, x):
        x = np.reshape(x, (len(x), 1))
        self.forward_pass(x)
        return self.a
    
    def load_a(self, a):
        self.a = a

In [14]:
#Cost function
def calc_cost(NN,x ,y):
    
    cost = 0
    for i in range(len(x)):
        x_ = np.reshape(x[i], (len(x[i]), 1))
        cost += 0.5 / len(x) * np.sum((y[i] - NN.forward_pass(x_)) ** 2)
    
    return cost

#Network initialization
autoencoder1 = NeuralNetwork([72, 60, 72])
autoencoder2 = NeuralNetwork([60,40,60])
autoencoder3 = NeuralNetwork([40, 30, 40])
NN = NeuralNetwork([72,60,40,30, 1])

In [15]:
#Autoencoder 1 pretraining
print("Autoencoder 1")
for i in range(200):
    for j, row in enumerate(X_train):
        row = np.reshape(row, (72,1))
        autoencoder1.train(row, row)
        
    cost = calc_cost(autoencoder1, X_train, X_train)
    if (i+1)%100 == 0:
        print(f"Epoch: {i+1}, Cost: {cost}")
    
#Scores computation for autoencoder 1
autoencoder2_input = []

for row in X_train:
    autoencoder2_input.append(autoencoder1.get_a(row)[1])

autoencoder2_input = np.array(autoencoder2_input)


#Autoencoder 2 pretraining
print("Autoencoder 2")
for i in range(200):
    for j, row in enumerate(autoencoder2_input):
        row = np.reshape(row, (60,1))
        autoencoder2.train(row, row)
        
    cost = calc_cost(autoencoder2, autoencoder2_input, autoencoder2_input)
    if (i+1)%100 == 0:
        print(f"Epoch: {i+1}, Cost: {cost}")


#Scores computation for autoencoder 2
autoencoder3_input = []

for row in autoencoder2_input:
    autoencoder3_input.append(autoencoder2.get_a(row)[1])

autoencoder3_input = np.array(autoencoder3_input)

#Autoencoder 3 pretraining
print("Autoencoder 3")
for i in range(200):
    for j, row in enumerate(autoencoder3_input):
        row = np.reshape(row, (40,1))
        autoencoder3.train(row, row)
        
    cost = calc_cost(autoencoder3, autoencoder3_input, autoencoder3_input)
    if (1+i)%100 == 0:
        print(f"Epoch: {i+1}, Cost: {cost}")

#Final network weight initialization
W1 = autoencoder1.get_W()[1]
W2 = autoencoder2.get_W()[1]
W3 = autoencoder3.get_W()[1]
W_final = {}
W_final[1] = W1
W_final[2] = W2
W_final[3] = W3
W_final[4] = np.random.randn(30, 1)
NN.load_W(W_final)

Autoencoder 1
Epoch: 100, Cost: 3216.9559953970356
Epoch: 200, Cost: 3209.1462642445617
Autoencoder 2
Epoch: 100, Cost: 4.78892496094285
Epoch: 200, Cost: 4.772383897367545
Autoencoder 3
Epoch: 100, Cost: 2.484828192250361
Epoch: 200, Cost: 2.4779127632846967


In [16]:
#Training
for i in range(500):
    if (i+1)%100==0:
        print("Epoch: ", i+1)

    for j in range(len(X_train)):
        NN.train(X_train[j], y_train[j])

y_pred = []
for i in range(len(X_val)):

    x = np.reshape(X_val[i], (len(X_val[i]), 1))
    x = NN.forward_pass(x)
    p = 0 if x[0] < 0.5 else 1
    y_pred.append(p)



Epoch:  100
Epoch:  200
Epoch:  300
Epoch:  400
Epoch:  500


In [17]:
confmat = cm(y_val, y_pred)

Accuracy = (confmat[0][0]+confmat[1][1])/len(y_pred)
Sensitivity = (confmat[1][1])/(confmat[1][0] + confmat[1][1])
Specificity = (confmat[0][0])/(confmat[0][0] + confmat[0][1])
print("Confusion Matrix:")
print(confmat)
print("\n")
print(f"Accuracy: {Accuracy*100}%\nSensitivity: {Sensitivity}\nSpecificity: {Specificity}\n")

Confusion Matrix:
[[264  49]
 [ 46 286]]


Accuracy: 85.27131782945736%
Sensitivity: 0.8614457831325302
Specificity: 0.8434504792332268

